<a href="https://colab.research.google.com/github/MK316/Getpp24/blob/main/GetPP_newanalysis0815.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0815 Data process revisited

+ Puncutation error in counting number of sentences and put them as a list
+ data to process: 00_textall.csv (text and text-only columns)

In [ ]:
import pandas as pd

# Replace 'yourfile.csv' with the actual name of your CSV file
# You can find the file name from the 'uploaded' dictionary if needed
df = pd.read_csv('/content/00_textall.csv', encoding='utf-8')

# Display the column names
print("Column names in the DataFrame:")
print(df.columns)


For the data, add space after sentence puncuations. Then save the file as a new one ('01_textall-space.csv')

Note: Solution 3: Use the chardet library to detect the file encoding automatically and load the file with the correct encoding.

In [ ]:
import pandas as pd
import re

# Load your CSV file (replace 'yourfile.csv' with the actual file name)
df = pd.read_csv('/content/00_textall.csv')

# Function to fix spacing after punctuation marks
def fix_spacing(text):
    # Use regex to add space after periods, exclamation marks, and question marks if there isn't any
    corrected_text = re.sub(r'([.!?])([A-Za-z])', r'\1 \2', text)
    return corrected_text

# Ensure that 'text-only' column is treated as string data
df['text-only'] = df['text-only'].astype(str)

# Apply the spacing correction function to the 'text-only' column and store it in 'text-corrected'
df['text-corrected'] = df['text-only'].apply(fix_spacing)

# Save the revised DataFrame with the corrected text to a new CSV file
df.to_csv('/content/01_textall-space.csv', index=False, encoding='utf-8')

# Optional: Download the CSV file to your local machine in Colab
from google.colab import files
files.download('/content/01_textall-space.csv')


# New Nsent: 'NumSent'

In [ ]:
data = pd.read_csv('/content/01_textall-space.csv', encoding='utf-8')

In [ ]:
# Display the column names
print("Column names in the DataFrame:")
print(df.columns)

### NewSentlist' and NumSent' columns

+ Then the data is saved as 02_textall-sentlist.csv

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

# Download the NLTK 'punkt' tokenizer data for sentence splitting
nltk.download('punkt')

# Load your CSV file (replace 'yourfile.csv' with the actual file name)
df = pd.read_csv('/content/01_textall-space.csv')

# Step 1: Ensure that 'text-corrected' column is treated as string data
df['text-corrected'] = df['text-corrected'].astype(str)

# Step 2: Split the text in 'text-corrected' into sentences and store in 'NewSentlist'
df['NewSentlist'] = df['text-corrected'].apply(sent_tokenize)

# Step 3: Count the number of sentences in each list and store in 'NumSent'
df['NumSent'] = df['NewSentlist'].apply(len)

# Step 4: Save the modified DataFrame to a new CSV file
df.to_csv('/content/02_textall-sentlist.csv', index=False, encoding='utf-8')

# Optional: Download the file to your local machine in Colab
from google.colab import files
files.download('/content/02_textall-sentlist.csv')


In [ ]:
df.head()

# Bepp vs. Getpp list and counts

Functions

In [ ]:
import re

# List of irregular past participles from the 100 most common English verbs
irregular_past_participles = [
    'been', 'become', 'begun', 'bent', 'bitten', 'blown', 'broken', 'brought', 'built',
    'bought', 'caught', 'chosen', 'come', 'cut', 'done', 'drawn', 'driven', 'eaten',
    'fallen', 'fed', 'felt', 'fought', 'found', 'flown', 'forgotten', 'forgiven',
    'frozen', 'gotten', 'given', 'gone', 'grown', 'hung', 'heard', 'hidden', 'held',
    'kept', 'known', 'laid', 'led', 'left', 'lent', 'let', 'lost', 'made', 'meant',
    'met', 'paid', 'put', 'read', 'ridden', 'risen', 'run', 'said', 'seen', 'sold',
    'sent', 'set', 'shaken', 'shone', 'shot', 'shown', 'shut', 'sung', 'sunk', 'sat',
    'slept', 'spoken', 'spent', 'stood', 'stolen', 'stuck', 'struck', 'sworn', 'swum',
    'taken', 'taught', 'torn', 'told', 'thought', 'thrown', 'understood', 'woken',
    'won', 'written'
]

# Convert the list of irregular past participles into a regex pattern
irregular_past_participles_pattern = r'\b(?:' + '|'.join(irregular_past_participles) + r')\b'

# Function to identify 'be' + past participle, including irregular verbs
def find_be_passive(sentences):
    be_passive_pattern = re.compile(
        r'\b(?:is|are|was|were|am|be|been|being|\'s)\s+(?:\w+ed|\w+en|' + irregular_past_participles_pattern + r')\b',
        re.IGNORECASE
    )
    return [sentence for sentence in sentences if be_passive_pattern.search(sentence)]

# Function to identify 'get' + past participle, including irregular verbs
def find_get_passive(sentences):
    get_passive_pattern = re.compile(
        r'\bget\s+(?:\w+ed|\w+en|' + irregular_past_participles_pattern + r')\b',
        re.IGNORECASE
    )
    return [sentence for sentence in sentences if get_passive_pattern.search(sentence)]


Count bepp and getpp

In [ ]:
df.head()

In [ ]:
# step 3

# Load the data
data = df

# Process each list of sentences
data['Bepp'] = data['NewSentlist'].apply(lambda x: find_be_passive(eval(x)))
data['Getpp'] = data['NewSentlist'].apply(lambda x: find_get_passive(eval(x)))

# Count the number of sentences for each passive form
data['Nbepp'] = data['Bepp'].apply(len)
data['Ngetpp'] = data['Getpp'].apply(len)


In [ ]:
# step 3

# Load the data
data = df

# Process each list of sentences
# Assuming 'NewSentlist' already contains lists of strings
data['Bepp'] = data['NewSentlist'].apply(find_be_passive)
data['Getpp'] = data['NewSentlist'].apply(find_get_passive)

# Count the number of sentences for each passive form
data['Nbepp'] = data['Bepp'].apply(len)
data['Ngetpp'] = data['Getpp'].apply(len)

In [ ]:
data.head()

# File to save (getpp-results-new.xlsx)

In [ ]:
import pandas as pd

# Save the DataFrame as an Excel file
data.to_excel('/content/getpp-results-new.xlsx', index=False)

# Optional: Download the file directly to your local machine (useful in Colab)
from google.colab import files
files.download('/content/getpp-results-new.xlsx')


--- Data process completed. Move to analysis